# **循环神经网络的从0实现**

In [1]:
import math
import numpy as np
import torch
from torch import nn, optim
import torch.nn.functional as F

import sys
sys.path.append('../utils')

In [2]:
import d2lzh as d2l
device = torch.device('cuda')
(corpus_indices, char_to_idx, idx_to_char, vocab_size) = d2l.load_data_jay_lyrics() # corpus_indices是语料库转换为id

## **one-hot向量**

In [3]:
def ont_hot(x, n_class, dtype=torch.float32):
    x = x.long()
    res = torch.zeros(x.shape[0], n_class, dtype=dtype, device=x.device)
    res.scatter_(1, x.view(-1, 1), 1)
    return res

关于scatter，传入的参数为(dim，index， src)，假设dim为1的话
```python
res[i][index[i][j]] = src
```

In [4]:
x = torch.tensor([0, 2])
ont_hot(x, 5)

tensor([[1., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0.]])

```python
res[0][index[0]] = src
res[1][index[2]] = src
```

In [5]:
def to_onehot(X, n_class):
    return [ont_hot(X[:, i], n_class) for i in range (X.shape[1])]

In [6]:
X = torch.arange(10).view(2, 5)

In [7]:
X

tensor([[0, 1, 2, 3, 4],
        [5, 6, 7, 8, 9]])

In [8]:
to_onehot(X, 10)

[tensor([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.]]),
 tensor([[0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.]]),
 tensor([[0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.]]),
 tensor([[0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.]]),
 tensor([[0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]])]

其实pytorch有提供自带的one-hot的方法

In [9]:
F.one_hot(X, 10).shape

torch.Size([2, 5, 10])

## **初始化模型参数**

In [10]:
num_inputs, num_hiddens, num_outputs = vocab_size, 256, vocab_size
print('will use', device)

def get_params():
    def _one(shape):
        ts = torch.tensor(np.random.normal(0, 0.01, size=shape), device=device, dtype=torch.float32)
        return torch.nn.Parameter(ts, requires_grad=True)
    
    # 隐藏层参数
    W_xh = _one((num_inputs, num_hiddens))
    W_hh = _one((num_hiddens, num_hiddens))
    b_h = torch.nn.Parameter(torch.zeros(num_hiddens, device=device, requires_grad=True))
    # 输出层参数
    W_hq = _one((num_hiddens, num_outputs))
    b_q = torch.nn.Parameter(torch.zeros(num_outputs, device=device, requires_grad=True))
    return nn.ParameterList([W_xh, W_hh, b_h, W_hq, b_q])

will use cuda


## **定义模型**

首先我们定义一个返回RNN初始状态的函数，RNN的初始状态被设定为全0的Array，我们为了方便初始状态可能存在多个Array的情况使用了元组的方式

In [11]:
def init_rnn_state(batch_size, num_hiddens, device):
    return (torch.zeros((batch_size, num_hiddens), device=device), )

下面的RNN函数定义了一个时间步内的计算，激活函数使用了tanh函数

In [12]:
def rnn(inputs, state, params):
    W_xh, W_hh, b_h, W_hq, b_q = params
    H, = state
    outputs = []
    for X in inputs:
        H = torch.tanh(torch.matmul(X, W_xh) + torch.matmul(H, W_hh) + b_h)
        Y = torch.matmul(H, W_hq) + b_q
        outputs.append(Y)
    return outputs, (H, ) #返回每个时间步的输出和最终的状态

In [14]:
# 测试, 时间步总共有5步， 1027是词表大小
state = init_rnn_state(X.shape[0], num_hiddens, device)
inputs = to_onehot(X.to(device), vocab_size)
params = get_params()
outputs, state_new = rnn(inputs, state, params)
print(len(outputs), outputs[0].shape, state_new[0].shape)

5 torch.Size([2, 2582]) torch.Size([2, 256])


## **预测函数**

这个函数在给定前面的内容，我们称之为prefix的情况下预测一定数量(num_chars)个字符，函数将循环神经单元rnn设置为了参数

In [15]:
def predict_rnn(prefix, num_chars, rnn, params, init_rnn_state, num_hiddens, vocab_size, device, idx_to_char, char_to_idx):
    state = init_rnn_state(1, num_hiddens, device)
    output = [char_to_idx[prefix[0]]]
    for t in range(num_chars + len(prefix) - 1):
        # 将上时间步的输出作为当前时间步的输入
        X = to_onehot(torch.tensor([[output[-1]]], device=device), vocab_size)
        # 计算输出和更新隐藏状态
        (Y, state) = rnn(X, state, params)
        # 下一个时间步是prefix里面的字符或者当前的最佳预测字符
        if t < len(prefix) - 1:
            output.append(char_to_idx[prefix[t + 1]])
        else:
            output.append(int(Y[0].argmax(dim=1).item()))
    return ''.join([idx_to_char[i] for i in output])

In [16]:
predict_rnn('分开', 10, rnn, params, init_rnn_state, num_hiddens, vocab_size,
            device, idx_to_char, char_to_idx)

'分开茉解奖暴莎捆洋肺〖绝'

## **梯度裁剪**

为了避免循环神经网络会出现的**梯度消失**和**梯度爆炸**的问题，我们需要做出梯度裁剪

梯度裁剪的方式有很多，这里我们把所有参数的梯度拼成一个向量$\boldsymbol g$，假设裁剪的阈值是$\theta$

$\large\min\left(\frac{\theta}{\|\boldsymbol{g}\|}, 1\right)\boldsymbol{g}$

保证$\boldsymbol g$的模不超过$\theta$

In [17]:
def grad_clipping(params, theta, device):
    norm = torch.tensor([0.0], device=device)
    for param in params:
        norm += (param.grad.data ** 2).sum()
    norm = norm.sqrt().item()
    if norm > theta:
        for param in params:
            param.grad.data *= (theta / norm)

## **定义模型训练函数**

In [18]:
def train_and_predict_rnn(rnn, get_params, init_rnn_state, num_hiddens, 
                          vocab_size, device, corpus_indices, idx_to_char,
                          char_to_idx, is_random_iter, num_epochs, num_steps,
                          lr, clipping_theta, batch_size, pred_period, pred_len, prefixes):
    if is_random_iter:
        data_iter_fn = d2l.data_iter_random
    else:
        data_iter_fn = d2l.data_iter_consecutive
    params = get_params()
    loss = nn.CrossEntropyLoss()
    first_step = True
    for epoch in range(num_epochs):
        if not is_random_iter: # 采用了相邻采样，在epoch开始时初始化隐藏状态
            state = init_rnn_state(batch_size, num_hiddens, device)
        l_sum, n = 0.0, 0
        data_iter = data_iter_fn(corpus_indices, batch_size, num_steps, device)
        for X, Y in data_iter:
            if is_random_iter: # 当使用随机采样，在每个batch之前进行初始化
                state = init_rnn_state(batch_size, num_hiddens, device)
            else:
                # detach会创建一个新的tensor，并和原来的计算图分离开来，data和原来的参数相同，但是require_grad变为false
                # 这里detach后，就不会对初始隐藏状态之前的内容求梯度，避免梯度计算太远
                for s in state:
                    s.detach_() # 将隐藏状态从计算图中分离处理，避免模型的梯度计算依赖过远计算开销太大
            inputs = to_onehot(X, vocab_size) # 将inputs转换为one-hot的形式
            (outputs, state) = rnn(inputs, state, params) # outputs包含num_steps个(batch_size, vocab_size)的矩阵
            outputs = torch.cat(outputs, dim=0) # (num_steps, batch_size, vocab_size)
            # Y的形状是(batch_size, num_steps)，要转置变成(num_steps, batch_size), 这样就和outputs一一对应
            y = torch.transpose(Y, 0, 1).contiguous().view(-1) # 当使用了transpose和permute再view最好加上contiguous(涉及到tensor的存储)
            # y : batch_size* num_steps outputs: batch_size*num_steps, vocab_size
            l = loss(outputs, y.long())
            if (not first_step) and (params[0].grad is not None):
                for param in params:
                    param.grad.data.zero_()
                first_step = False
            l.backward()
            grad_clipping(params, clipping_theta, device)
            d2l.sgd(params, lr, 1)
            l_sum += l.item() * y.shape[0]
            n += y.shape[0]
        if (epoch + 1) % pred_period == 0:
            print(f'epoch{epoch + 1}, perplexity:{math.exp(l_sum / n):.4f}')
            for prefix in prefixes:
                print('-', predict_rnn(prefix, pred_len, rnn, params, init_rnn_state,
                    num_hiddens, vocab_size, device, idx_to_char, char_to_idx))

## **训练**

In [19]:
num_epochs, num_steps, batch_size, lr, clipping_theta = 250, 35, 32, 1e2, 1e-2
pred_period, pred_len, prefixes = 50, 50, ['有没', '我只']

In [20]:
train_and_predict_rnn(rnn, get_params, init_rnn_state, num_hiddens,
                      vocab_size, device, corpus_indices, idx_to_char,
                      char_to_idx, True, num_epochs, num_steps, lr,
                      clipping_theta, batch_size, pred_period, pred_len,
                      prefixes) # 随机采样

epoch50, perplexity:39.6650
- 有没有天 只能不觉 我不需要我 我不能再想 我不能再想 我不能再想 我不能再想 我不能再想 我不能再想 
- 我只能在远 一统一步 你有一场我想要你 你我的笑我 你已经着你 我用眼你的微笑 让我想 你不过 这样的太
epoch100, perplexity:15.1668
- 有没有结光 我们的身 在一起 梦 一统上 木生的 月在我的地球   我想 功不需 重 再太快的 用时间 
- 我只是你 你我已笑我 你我很路的相样 我们的地一路 一些长色的面 只让我们遇见 爱你的人 在远窗的面 在
epoch150, perplexity:9.3204
- 有没有人知道  分用说就是别是停我的照   其里  一个人的微笑  你说  麦理  我说我选装的裁 我用
- 我只是你像一句幕 我知道走好要离开 我要我这种爱 我没是你说都是多 再想我不出你 不知想觉太快手我该要 
epoch200, perplexity:6.6654
- 有没有忘色 心 我没人拥 我坚拳看得到 这里的山门　 我在想醒上降人 是他们失无之来 我想要被你看着 我
- 我只是会再你 你们一起你说不到我 我知道着这没有理 还不会给 我们将等你离 冰筆乱 的世魂 翻滚的人倾 
epoch250, perplexity:5.0358
- 有没有人知道 你的那笑像拥抱我却有人功夫  我这第一个很美 如果说对 你是否了我 这个街色 跨大地心 我
- 我只是一种开了 家色的象被 风刺亮的路寂 感情 带止忿婪 我在街里 历史轮回 转身忘掉  感前的人像 幸


In [21]:
pred_period, pred_len, prefixes = 50, 50, ['思念', '还记得']
train_and_predict_rnn(rnn, get_params, init_rnn_state, num_hiddens,
                      vocab_size, device, corpus_indices, idx_to_char,
                      char_to_idx, False, num_epochs, num_steps, lr,
                      clipping_theta, batch_size, pred_period, pred_len,
                      prefixes)

epoch50, perplexity:37.3506
- 思念 不用你 我一种 北尽的山 我在我的爱了 你的你爱在远   我  功你的选 有一种 梦叫一种 你的手
- 还记得 你的爱魂 我不能感开  我知道这不要 我的你 再不该 让我的人手 你说你离开 你的爱后 你说你的泪
epoch100, perplexity:17.4790
- 思念的美 再开名受   用  有你 就坚                                
- 还记得 你心一起 你的手 再魂 我哭 不想就是 为就能不用                        
epoch150, perplexity:11.7575
- 思念的美  你说一笔 一起两 告炭的话月 不让  一个放  你的话尖 我们无悔同一生 你微笑的话样 在空
- 还记得 你的灵魂不要 我一路就自己的甜 你说的爱情笑 我要的黑样 雨来不明 你的手 将你的完泪 她后 我只
epoch200, perplexity:8.5939
- 思念你的爱情 你好你的爱写在西元前 深埋在美索不达米亚平原 几楔形世字染下好局  所事老的眼头 你都微好
- 还记得太多的我会不算你好微 为专的歌息成了 不懂在薄恼 我用儿有些人 我知道你的笑 有一种会道叫你 不到她
epoch250, perplexity:6.6780
- 思念你 我身了几 回份的了解  我的眼尖 你的回忆 我已无的伤 再去的香平 我们往悔 一麻续杯 分手一直
- 还记得 我怎么没有个它 能明月 我说你也不了 你说的爱 太你的香 我们  我们武 像想都没有深 我听你打开


## **补充资料:自然语言中的困惑度**